# cobrapy

### Installation: 
Note: I am using OS X Yosemite, Python 2.7.11, with Anaconda 2.4.1

1. If you are using an OS with Anaconda, first download and install glpk separately with this command in shell
 - conda install -c https://conda.anaconda.org/mcg glpk
2. Then install cobrapy from shell by using
 - pip install cobra
 
 https://cobrapy.readthedocs.org/en/stable/getting_started.html

### Basics

In [1]:
from __future__ import print_function
import cobra.test

model = cobra.test.create_test_model("salmonella")  # "ecoli" and "salmonella" are also valid arguments

cobra/io/__init__.py:17 UserWarning: cobra.io.sbml requires libsbml


In [2]:
print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))

2546
1802
1264


In [3]:
model.reactions[29]

<Reaction 2AGPA180tipp at 0x10807a3d0>

In [4]:
model.metabolites.get_by_id("atp_c")

<Metabolite atp_c at 0x107dd4850>

In [5]:
model.reactions.EX_glc__D_e.lower_bound

0.0

In [6]:
pgi = model.reactions.get_by_id("PGI")
pgi

<Reaction PGI at 0x1093c7c10>

In [7]:
print(pgi.name)
print(pgi.reaction)

glucose 6 phosphate isomerase
g6p_c <=> f6p_c


In [8]:
print(pgi.lower_bound, "< pgi <", pgi.upper_bound)
print(pgi.reversibility)

-1000.0 < pgi < 1000.0
True


In [9]:
pgi.check_mass_balance()

[]

In [10]:
pgi.add_metabolites({model.metabolites.get_by_id("h_c"): -1})
pgi.reaction

'h_c + g6p_c <=> f6p_c'

In [11]:
pgi.check_mass_balance()

['PGI', {'C': 0.0, 'H': -1.0, 'O': 0.0, 'P': 0.0}]

In [12]:
pgi.pop(model.metabolites.get_by_id("h_c"))
print(pgi.reaction)
print(pgi.check_mass_balance())

g6p_c <=> f6p_c
[]


In [13]:
pgi.reaction = "g6p_c --> f6p_c + h_c + green_eggs + ham"

unknown metabolite 'green_eggs' created
unknown metabolite 'ham' created


In [14]:
pgi.reaction

'g6p_c --> h_c + ham + green_eggs + f6p_c'

### simulating with FBA

In [15]:
import cobra.test
model = cobra.test.create_test_model()

In [16]:
model.optimize()

<Solution 0.38 at 0x109b47b10>

In [17]:
model.solution.status

'optimal'

In [18]:
model.solution.f

0.38000797224652744

##### Changing the objective

In [19]:
{reaction: reaction.objective_coefficient for reaction in model.reactions
        if reaction.objective_coefficient > 0}

{<Reaction biomass_iRR1083_metals at 0x10af19150>: 1.0}

In [20]:
# change the objective to ATPM
# the upper bound should be 1000 so we get the actual optimal value
model.reactions.get_by_id("ATPM").upper_bound = 1000.
model.change_objective("ATPM")
{reaction: reaction.objective_coefficient for reaction in model.reactions
        if reaction.objective_coefficient > 0}

{<Reaction ATPM at 0x109daef50>: 1.0}

In [21]:
model.optimize()

<Solution 119.67 at 0x10af19c90>

In [22]:
model.reactions.get_by_id("ATPM").objective_coefficient = 0.
model.reactions.get_by_id("biomass_iRR1083_metals").objective_coefficient = 1.
{reaction: reaction.objective_coefficient for reaction in model.reactions
        if reaction.objective_coefficient > 0}

{<Reaction biomass_iRR1083_metals at 0x10af19150>: 1.0}

In [23]:
model.optimize()

<Solution 0.38 at 0x10af19e50>

### Simulating deletions

In [24]:
from time import time


from cobra.test import create_test_model, salmonella_pickle, ecoli_pickle
from cobra.flux_analysis import single_deletion
from cobra.flux_analysis import double_deletion


cobra_model = create_test_model(salmonella_pickle)

In [25]:
start = time()  # start timer()
growth_rates, statuses = single_deletion(cobra_model) # deletes single genes
print("All single gene deletions completed in %.2f sec" % (time() - start))

All single gene deletions completed in 4.48 sec


In [26]:
single_deletion(cobra_model, element_list=cobra_model.genes[:100]);

In [27]:
start = time()  # start timer()
growth_rates, statuses = single_deletion(cobra_model, element_type="reaction") # deletes single reactions
print("All single reaction deletions completed in %.2f sec" % (time() - start))

All single reaction deletions completed in 8.36 sec


In [28]:
start = time()  # start timer()
double_deletion(cobra_model, element_list_1=cobra_model.genes[:100]) # double deletions
print("Double gene deletions for 100 genes completed in %.2f sec" % (time() - start))

Double gene deletions for 100 genes completed in 8.28 sec


In [29]:
cobra_model.element_list

AttributeError: 'Model' object has no attribute 'element_list'

In [ ]:
# if panda is installed

frame = double_deletion(cobra_model, element_list_1=cobra_model.reactions[300:308],
                        element_type="reaction", return_frame=True)
frame[frame < 1e-9] = 0.  # round small values to 0
frame

## Reading and Writing outputs (JSON)

In [ ]:
import cobra.test

print("E. coli test files: ")
print(", ".join([i for i in dir(cobra.test) if i.startswith("ecoli")]))
print("")
print("Salmonella test files: ")
print(", ".join([i for i in dir(cobra.test) if i.startswith("salmonella")]))

salmonella_model = cobra.test.create_test_model("salmonella")

In [ ]:
cobra.io.load_json_model(cobra.test.ecoli_json)

### *This saves the model and outputs it as a json file*

In [ ]:
#cobra.io.save_json_model(salmonella_model, "test.json")